# Agents processing
1) Generate a subset of agents who live within Västra Gätalands Region.

In [1]:
%load_ext autoreload
%autoreload 2
%cd D:\synthetic-sweden

D:\synthetic-sweden


In [2]:
# Load libs
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import dataworkers as dw

## Load boundary

In [3]:
boundary = gpd.read_file('dbs/municipalities/sweden_komk.shp')
mun_in_boundary = list(boundary.loc[boundary['county'] == 'Västra Götaland', 'KnKod'].values)

## Load agents and their daily plans

In [4]:
df_pop = pd.read_pickle('dbs/agents/syn_pop_all.pkl')
df_pop.head(3)

,PId,Deso,kommun,marital,sex,age,HId,HHtype,HHsize,num_babies,employment,studenthood,income_class,num_cars,HHcars,pot_car_driver
0,1,0115A0040,0115,couple,1,22,1,couple,2,0,1,0,1,0,0,0.279179
1,2,0115A0040,0115,couple,0,20,1,couple,2,0,0,1,0,0,0,0.044359
2,3,0115A0040,0115,couple,0,21,2,couple,2,0,1,1,2,1,2,0.068788


In [5]:
df_pop_sub = df_pop.loc[df_pop['kommun'].isin(mun_in_boundary), :]
print("Agents live in Västra Götalands: %s"%len(df_pop_sub))

Agents live in Västra Götalands: 1717772


In [6]:
df_plan = pd.read_pickle('dbs/agents/df_act_plan.pkl')
df_plan.head(3)

,PId,act_id,joint_PId_actid,act_purpose,act_start,act_end,mode,building_ID,building_type,Deso,Reg_model,New_Zone_ID,POINT_X_sweref99,POINT_Y_sweref99,POINT_X,POINT_Y
0,1,0,1;0,1,3.000000,7.293010,,4170405,130,0115A0040,Samm,3711513,684203.6685,6.617561e+06,18.269388,59.655295
1,1,1,1;1,4,7.762524,13.693192,CarPassenger,3725941,399,0180C3560,Samm,3720172,670042.8868,6.579364e+06,17.988031,59.318781
2,1,2,1;2,10,14.162705,16.888355,CarPassenger,3749926,499,0180C3090,Samm,3710407,670286.4225,6.578486e+06,17.991611,59.310809


In [7]:
df_plan_sub = df_plan.loc[df_plan['PId'].isin(df_pop_sub.PId), :]
print("Agents live in Västra Götalands have activities: %s"%len(df_plan_sub))

Agents live in Västra Götalands have activities: 7009790


## Keep agents and their activities if they all locate within the boundary
This is not necessary anymore for the current version of case study.

In [37]:
gdf = dw.df2gdf_point(df_plan_sub, 'POINT_X', 'POINT_Y', crs=4326, drop=False)
gdf.loc[:, 'boundary'] = gdf.apply(lambda row: boundary.contains(row.geometry), axis=1)
gdf.head(2)

,PId,act_id,joint_PId_actid,act_purpose,act_start,act_end,mode,building_ID,building_type,Deso,Reg_model,New_Zone_ID,POINT_X_sweref99,POINT_Y_sweref99,POINT_X,POINT_Y,geometry,boundary
4421287,1044305,0,1044305;0,1,3.000000,7.863121,,353594,130,0643C1030,National,1986231,446134.3136,6.418626e+06,14.091069,57.906175,POINT (14.09107 57.90617),False
4421288,1044305,1,1044305;1,4,7.954329,11.771308,Car,358719,699,0643C1010,National,1986231,447177.8607,6.417934e+06,14.108829,57.900082,POINT (14.10883 57.90008),False


In [63]:
tqdm.pandas()
df_id2select = gdf.groupby('PId').progress_apply(lambda g: all(g['boundary'])).reset_index(name='select')
id2select = df_id2select.loc[df_id2select['select'], 'PId'].values

100%|██████████| 1717772/1717772 [12:07<00:00, 2361.53it/s]


## Save agents and their plans for a subset in VRG

In [8]:
df_plan_sub.to_pickle("dbs/agents/df_act_plan_vgr.pkl")
df_pop_sub.to_pickle("dbs/agents/syn_pop_vgr.pkl")